# Building a Lead Scoring Model with EvalML

In this demo, we will build an optimized lead scoring model using EvalML. To optimize the pipeline, we will set up an objective function to maximize the revenue generated with true positives while taking into account the cost of false positives. At the end of this demo, we also show you how introducing the right objective during the training is significantly better than using a generic machine learning metric like AUC.

In [1]:
import evalml
from evalml import AutoMLSearch
from evalml.objectives import LeadScoring

## Configure LeadScoring 

To optimize the pipelines toward the specific business needs of this model, you can set your own assumptions for how much value is gained through true positives and the cost associated with false positives. These parameters are

* `true_positive` - dollar amount to be gained with a successful lead
* `false_positive` - dollar amount to be lost with an unsuccessful lead

Using these parameters, EvalML builds a pileline that will maximize the amount of revenue per lead generated.

In [2]:
lead_scoring_objective = LeadScoring(
    true_positives=50,
    false_positives=-5
)

## Dataset

We will be utilizing a dataset detailing a customer's job, country, state, zip, online action, the dollar amount of that action and whether they were a successful lead.

In [3]:
from urllib.request import urlopen
import pandas as pd
import woodwork as ww
customers_data = urlopen('https://featurelabs-static.s3.amazonaws.com/lead_scoring_ml_apps/customers.csv')
interactions_data = urlopen('https://featurelabs-static.s3.amazonaws.com/lead_scoring_ml_apps/interactions.csv')
leads_data = urlopen('https://featurelabs-static.s3.amazonaws.com/lead_scoring_ml_apps/previous_leads.csv')
customers = pd.read_csv(customers_data)
interactions = pd.read_csv(interactions_data)
leads = pd.read_csv(leads_data)

X = customers.merge(interactions, on='customer_id').merge(leads, on='customer_id')
y = X['label']
X = X.drop(['customer_id', 'date_registered', 'birthday','phone', 'email',
            'owner', 'company', 'id', 'time_x',
            'session', 'referrer', 'time_y', 'label', 'country'], axis=1)
display(X.head())

,job,state,zip,action,amount
0,"Engineer, mining",NY,60091.0,page_view,NaN
1,"Psychologist, forensic",CA,NaN,purchase,135.23
2,"Psychologist, forensic",CA,NaN,page_view,NaN
3,Air cabin crew,NaN,60091.0,download,NaN
4,Air cabin crew,NaN,60091.0,page_view,NaN


We will convert our data into Woodwork data structures. Doing so enables us to have more control over the types passed to and inferred by AutoML.

In [4]:
X.ww.init(semantic_tags={'job': 'category'}, logical_types={'job': 'Categorical'})
y = ww.init_series(y)
X.ww

,Physical Type,Logical Type,Semantic Tag(s)
Column,,,
job,category,Categorical,['category']
state,category,Categorical,['category']
zip,float64,Double,['numeric']
action,category,Categorical,['category']
amount,float64,Double,['numeric']


## Search for the best pipeline

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set.

EvalML natively supports one-hot encoding and imputation so the above `NaN` and categorical values will be taken care of.

In [5]:
X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(X, y, problem_type='binary', test_size=0.2, random_seed=0)

X.ww

,Physical Type,Logical Type,Semantic Tag(s)
Column,,,
job,category,Categorical,['category']
state,category,Categorical,['category']
zip,float64,Double,['numeric']
action,category,Categorical,['category']
amount,float64,Double,['numeric']


Because the lead scoring labels are binary, we will use set the problem type to "binary". When we call `.search()`, the search for the best pipeline will begin. 

In [6]:
automl = AutoMLSearch(X_train=X_train, y_train=y_train,
                      problem_type='binary',
                      objective=lead_scoring_objective,
                      additional_objectives=['auc'],
                      allowed_model_families=["catboost", "random_forest", "linear_model"],
                      max_batches=1,
                      verbose=True)

automl.search()

Generating pipelines to search over...


DEBUG:evalml.automl.automl_algorithm.iterative_algorithm.verbose:allowed_estimators set to ['Elastic Net Classifier', 'CatBoost Classifier', 'Random Forest Classifier', 'Logistic Regression Classifier']
DEBUG:evalml.automl.automl_algorithm.iterative_algorithm.verbose:allowed_pipelines set to ['Elastic Net Classifier w/ Label Encoder + Imputer + One Hot Encoder + Oversampler + Standard Scaler', 'Logistic Regression Classifier w/ Label Encoder + Imputer + One Hot Encoder + Oversampler + Standard Scaler', 'CatBoost Classifier w/ Label Encoder + Imputer + Oversampler', 'Random Forest Classifier w/ Label Encoder + Imputer + One Hot Encoder + Oversampler']
DEBUG:evalml.automl.automl_algorithm.iterative_algorithm.verbose:allowed_model_families set to ['catboost', 'random_forest', 'linear_model']


4 pipelines ready for search.


INFO:evalml.automl.automl_algorithm.iterative_algorithm.verbose:4 pipelines ready for search.



*****************************


...

INFO:evalml.automl.automl_search.verbose:
Search finished after 00:14            


Best pipeline: CatBoost Classifier w/ Label Encoder + Imputer + Oversampler


INFO:evalml.automl.automl_search.verbose:Best pipeline: CatBoost Classifier w/ Label Encoder + Imputer + Oversampler


Best pipeline Lead Scoring: 1.551920


INFO:evalml.automl.automl_search.verbose:Best pipeline Lead Scoring: 1.551920


### View rankings and select pipeline

Once the fitting process is done, we can see all of the pipelines that were searched, ranked by their score on the lead scoring objective we defined.

In [7]:
automl.rankings

,id,pipeline_name,search_order,mean_cv_score,standard_deviation_cv_score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,3,CatBoost Classifier w/ Label Encoder + Imputer...,3,1.551920,0.110441,1.551920,inf,False,"{'Label Encoder': {'positive_label': None}, 'I..."
1,4,Random Forest Classifier w/ Label Encoder + Im...,4,0.140912,0.125842,0.140912,inf,False,"{'Label Encoder': {'positive_label': None}, 'I..."
2,2,Logistic Regression Classifier w/ Label Encode...,2,0.132316,0.125820,0.132316,inf,False,"{'Label Encoder': {'positive_label': None}, 'I..."
3,1,Elastic Net Classifier w/ Label Encoder + Impu...,1,0.131241,0.126248,0.131241,inf,False,"{'Label Encoder': {'positive_label': None}, 'I..."
4,0,Mode Baseline Binary Classification Pipeline,0,0.000000,0.000000,0.000000,0.0,False,"{'Label Encoder': {'positive_label': None}, 'B..."


To select the best pipeline we can call `automl.best_pipeline`.

In [8]:
best_pipeline = automl.best_pipeline

### Describe pipeline

You can get more details about any pipeline, including how it performed on other objective functions by calling `.describe_pipeline()` and specifying the `id` of the pipeline.

In [9]:
automl.describe_pipeline(automl.rankings.iloc[0]["id"])


****************************************************************


INFO:evalml.pipelines.pipeline_base.describe:
****************************************************************


* CatBoost Classifier w/ Label Encoder + Imputer + Oversampler *


INFO:evalml.pipelines.pipeline_base.describe:* CatBoost Classifier w/ Label Encoder + Imputer + Oversampler *


****************************************************************


...

INFO:evalml.automl.automl_search.describe_pipeline:Cross Validation


----------------


INFO:evalml.automl.automl_search.describe_pipeline:----------------


             Lead Scoring   AUC # Training # Validation
0                   1.581 0.869      3,099        1,550
1                   1.645 0.887      3,099        1,550
2                   1.430 0.889      3,100        1,549
mean                1.552 0.882          -            -
std                 0.110 0.011          -            -
coef of var         0.071 0.012          -            -


INFO:evalml.automl.automl_search.describe_pipeline:             Lead Scoring   AUC # Training # Validation
0                   1.581 0.869      3,099        1,550
1                   1.645 0.887      3,099        1,550
2                   1.430 0.889      3,100        1,549
mean                1.552 0.882          -            -
std                 0.110 0.011          -            -
coef of var         0.071 0.012          -            -


## Evaluate on hold out

Finally, since the best pipeline was trained on all of the training data, we evaluate it on the holdout dataset.

In [10]:
best_pipeline_score = best_pipeline.score(X_holdout, y_holdout, objectives=["auc", lead_scoring_objective])
best_pipeline_score

OrderedDict([('AUC', 0.8585599879117558), ('Lead Scoring', 1.590713671539123)])

## Why optimize for a problem-specific objective?

To demonstrate the importance of optimizing for the right objective, let's search for another pipeline using AUC, a common machine learning metric. After that, we will score the holdout data using the lead scoring objective to see how the best pipelines compare.

In [11]:
automl_auc = evalml.AutoMLSearch(X_train=X_train, y_train=y_train,
                                 problem_type='binary',
                                 objective='auc',
                                 additional_objectives=[lead_scoring_objective],
                                 allowed_model_families=["catboost", "random_forest", "linear_model"],
                                 max_batches=1,
                                 verbose=True)

automl_auc.search()

Generating pipelines to search over...


INFO:evalml.automl.automl_algorithm.iterative_algorithm.verbose:Generating pipelines to search over...
DEBUG:evalml.automl.automl_algorithm.iterative_algorithm.verbose:allowed_estimators set to ['Elastic Net Classifier', 'CatBoost Classifier', 'Random Forest Classifier', 'Logistic Regression Classifier']
DEBUG:evalml.automl.automl_algorithm.iterative_algorithm.verbose:allowed_pipelines set to ['Elastic Net Classifier w/ Label Encoder + Imputer + One Hot Encoder + Oversampler + Standard Scaler', 'Logistic Regression Classifier w/ Label Encoder + Imputer + One Hot Encoder + Oversampler + Standard Scaler', 'CatBoost Classifier w/ Label Encoder + Imputer + Oversampler', 'Random Forest Classifier w/ Label Encoder + Imputer + One Hot Encoder + Oversampler']
DEBUG:evalml.automl.automl_algorithm.iterative_algorithm.verbose:allowed_model_families set to ['catboost', 'random_forest', 'linear_model']


4 pipelines ready for search.


INFO:evalml.automl.automl_algorithm.iterative_algorithm.verbose:4 pipelines ready for search.



*****************************


...

INFO:evalml.automl.automl_search.verbose:
Search finished after 00:12            


Best pipeline: CatBoost Classifier w/ Label Encoder + Imputer + Oversampler


INFO:evalml.automl.automl_search.verbose:Best pipeline: CatBoost Classifier w/ Label Encoder + Imputer + Oversampler


Best pipeline AUC: 0.881709


INFO:evalml.automl.automl_search.verbose:Best pipeline AUC: 0.881709


In [12]:
automl_auc.rankings

,id,pipeline_name,search_order,mean_cv_score,standard_deviation_cv_score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,3,CatBoost Classifier w/ Label Encoder + Imputer...,3,0.881709,0.010861,0.881709,38.170947,False,"{'Label Encoder': {'positive_label': None}, 'I..."
1,2,Logistic Regression Classifier w/ Label Encode...,2,0.652938,0.035827,0.652938,15.293849,False,"{'Label Encoder': {'positive_label': None}, 'I..."
2,1,Elastic Net Classifier w/ Label Encoder + Impu...,1,0.652736,0.035854,0.652736,15.273584,False,"{'Label Encoder': {'positive_label': None}, 'I..."
3,4,Random Forest Classifier w/ Label Encoder + Im...,4,0.651591,0.059622,0.651591,15.159098,False,"{'Label Encoder': {'positive_label': None}, 'I..."
4,0,Mode Baseline Binary Classification Pipeline,0,0.500000,0.000000,0.500000,0.000000,False,"{'Label Encoder': {'positive_label': None}, 'B..."


Like before, we can look at the rankings and pick the best pipeline.

In [13]:
best_pipeline_auc = automl_auc.best_pipeline

In [14]:
# get the auc and lead scoring score on holdout data
best_pipeline_auc_score = best_pipeline_auc.score(X_holdout, y_holdout,  objectives=["auc", lead_scoring_objective])
best_pipeline_auc_score

OrderedDict([('AUC', 0.8585599879117558), ('Lead Scoring', 1.590713671539123)])

In [15]:
assert best_pipeline_score['Lead Scoring'] >= best_pipeline_auc_score['Lead Scoring']
assert best_pipeline_auc_score['Lead Scoring'] >= 0

When we optimize for AUC, we can see that the AUC score from this pipeline is similar to the AUC score from the pipeline optimized for lead scoring. However, the revenue per lead is much smaller per lead when optimized for AUC and was much larger when optimized for lead scoring. As a result, we would have a huge gain on the amount of revenue if we optimized for lead scoring.

This happens because optimizing for AUC does not take into account the user-specified `true_positive` (dollar amount to be gained with a successful lead) and `false_positive` (dollar amount to be lost with an unsuccessful lead) values. Thus, the best pipelines may produce the highest AUC but may not actually generate the most revenue through lead scoring.

This example highlights how performance in the real world can diverge greatly from machine learning metrics.